In [2]:
import sqlalchemy
import pandas as pd

In [3]:
from sqlalchemy.engine import URL
connection_url = URL.create(
    "mssql+pyodbc",
    username="rudolfs",
    password="aaaa",
    host="localhost",
    port=1433,
    database="balta_bronze",
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "TrustServerCertificate": "yes",
    },
)
# engine = sqlalchemy.create_engine('mssql://localhost/balta_bronze?trusted_connection=yes')
connection_url
engine = sqlalchemy.create_engine(connection_url)

In [ ]:
import pandera.pandas as pa

In [40]:
# Example of bad data filtering via pandas
with engine.connect() as connection:
    claims = pd.read_sql(f"SELECT *  FROM balta_bronze.dbo.claims",connection)
    policies = pd.read_sql(f"SELECT *  FROM balta_bronze.dbo.policies",connection)
    df = claims.join(policies.set_index('policy_id').add_suffix('_policie_cols'), on='policy_id', how='left')
    df = df.loc[df['claim_date'] <= df['end_date_policie_cols']]
    df = df.loc[:,~df.columns.str.endswith('_policie_cols')]

    df.drop_duplicates(subset=['claim_id'], keep='first', inplace=True, ignore_index=True)
    df = df.reset_index(drop=True)

    connection.execute(text('USE [balta_silver];'))
    df.to_sql(name='claims',schema='dbo',con=connection,if_exists='replace',index=False)
    

In [33]:
# Example of bad data filtering via pandera
from datetime import datetime
import pandera.pandas as pa
import numpy as np 
from sqlalchemy import text
with engine.connect() as connection:
    df = pd.read_sql(f"SELECT *  FROM balta_bronze.dbo.offers",connection
                    )
    df = df.drop_duplicates(subset=['offer_id'], keep='first', ignore_index=True)
    df = df.reset_index(drop=True)
    schema = pa.DataFrameSchema(
        {"offer_id": pa.Column(str),
        "customer_id": pa.Column(int, pa.Check(lambda x: x > 0)),
        "product_code": pa.Column(str),
        "product_variant": pa.Column(str),
        "offer_datetime": pa.Column(pa.DateTime, pa.Check(lambda x: x < datetime.today())),
        "premium_offered": pa.Column(np.float64, pa.Check(lambda x: x > 0)),
        "sales_source": pa.Column(str),
        "sum_insured": pa.Column(np.float64, pa.Check(lambda x: x > 0)),
        "coverage_hash": pa.Column(str, pa.Check(lambda x: len(x) == 32))},
                index=pa.Index(int),
                drop_invalid_rows=True,
                coerce =True,
        )
    df = schema(df, lazy=True)    

    connection.execute(text('USE [balta_silver];'))
    df.to_sql(name='offers',schema='dbo',con=connection,if_exists='replace',index=False)

In [38]:
with engine.connect() as connection:
    df = pd.read_sql(f"SELECT *  FROM balta_bronze.dbo.policies",connection)
    df.drop_duplicates(subset=['policy_id'], keep='first', inplace=True, ignore_index=True)
    df = df.reset_index(drop=True)

    connection.execute(text('USE [balta_silver];'))
    df.to_sql(name='policies',schema='dbo',con=connection,if_exists='replace',index=False)

In [39]:

with engine.connect() as connection:
    df = pd.read_sql(f"SELECT *  FROM balta_bronze.dbo.customers",connection)
    df.drop_duplicates(subset=['customer_id'], keep='first', inplace=True, ignore_index=True)
    df = df.reset_index(drop=True)

    connection.execute(text('USE [balta_silver];'))
    df.to_sql(name='customers',schema='dbo',con=connection,if_exists='replace',index=False)